# **Features engineering**

In [7]:
import pandas as pd

In [8]:
train = pd.read_csv(r'data/train_final.csv')

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1441 entries, 0 to 1440
Columns: 250 entries, Id to Zoning_Neighborhood_RM_Sawyer
dtypes: float64(8), int64(241), object(1)
memory usage: 2.7+ MB


In [12]:
# Ver cuántos nulos hay en cada columna
null_counts = train.isnull().sum()

# Filtrar solo columnas con al menos un nulo
nulls_present = null_counts[null_counts > 0].sort_values(ascending=False)

# Mostrar
print("🔍 Columnas con valores nulos:")
print(nulls_present)


🔍 Columnas con valores nulos:
GarageYrBlt    81
dtype: int64


In [ ]:
# Filtrar registros con GarageYrBlt nulo
garageyrblt_nulls = train[train['GarageYrBlt'].isnull()]

# Ver sus valores en otras columnas relacionadas al garage
garage_check = garageyrblt_nulls[['GarageType_Attchd', 'GarageType_Detchd', 'GarageArea', 'GarageCars']]

# Mostrar primeras filas
print(garage_check.head(10))
print('Hay',len(garage_check),'registros sin garage')

# Ver cuántas tienen GarageArea > 0
print("\nCantidad con GarageArea > 0:", (garageyrblt_nulls['GarageArea'] > 0).sum())

# Y cuántas tienen GarageCars > 0
print("Cantidad con GarageCars > 0:", (garageyrblt_nulls['GarageCars'] > 0).sum())


     GarageType_Attchd  GarageType_Detchd  GarageArea  GarageCars
39                   0                  0           0           0
48                   0                  0           0           0
77                   0                  0           0           0
87                   0                  0           0           0
88                   0                  0           0           0
98                   0                  0           0           0
107                  0                  0           0           0
124                  0                  0           0           0
126                  0                  0           0           0
139                  0                  0           0           0
hay 81 registros sin garage

Cantidad con GarageArea > 0: 0
Cantidad con GarageCars > 0: 0


## 📊 Selección de Variables Numéricas Fuertemente Correlacionadas con `SalePrice`

🎯 **Objetivo**: Seleccionar automáticamente las variables numéricas que presentan una alta correlación con `SalePrice`, para usarlas luego en una expansión polinómica con `PolynomialFeatures`.

📌 **Criterios de selección**:
- Variables **numéricas continuas** (float o int)
- Correlación absoluta con `SalePrice` **mayor a 0.4**
- Se excluyen columnas target, IDs y categóricas codificadas

In [10]:
import numpy as np
import pandas as pd

# Asegurarse de tener solo variables numéricas
numeric_cols = train.select_dtypes(include=['int64', 'float64']).copy()

# Excluir la variable target y cualquier ID
excluded = ['SalePrice', 'Id']
numeric_cols = numeric_cols.drop(columns=[col for col in excluded if col in numeric_cols.columns])

# Calcular correlación con la variable target
correlations = numeric_cols.corrwith(train['SalePrice'])

# Filtrar por correlación fuerte
strong_features = correlations[correlations.abs() > 0.4].sort_values(ascending=False)

# Mostrar resultado
print("📈 Variables numéricas con correlación fuerte (> 0.4) con SalePrice:")
print(strong_features)

# Guardar lista de features para usar en PolynomialFeatures
selected_features = strong_features.index.tolist()


📈 Variables numéricas con correlación fuerte (> 0.4) con SalePrice:
OverallQual          0.823505
GrLivArea            0.733793
GarageCars           0.685087
ExterQual            0.681659
GarageArea           0.668695
KitchenQual          0.668332
TotalBsmtSF          0.639330
BsmtQual             0.614882
GarageFinish         0.606831
1stFlrSF             0.606006
FullBath             0.598591
YearBuilt            0.588787
YearRemodAdd         0.568595
GarageYrBlt          0.548012
FireplaceQu          0.540808
Foundation_PConc     0.538138
TotRmsAbvGrd         0.536313
Fireplaces           0.484236
HeatingQC            0.477931
BsmtFinType1_GLQ     0.441554
MasVnrArea           0.429103
GarageType_Attchd    0.414782
dtype: float64


## 🔁 Expansión Polinómica de Features Seleccionadas

🎯 **Objetivo**: Generar nuevas variables que representen combinaciones cuadráticas entre las features más correlacionadas con `SalePrice`, usando `PolynomialFeatures`.

📌 **Características del proceso**:
- Se seleccionaron automáticamente variables numéricas con |correlación| > 0.4
- Se aplicó `PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)`
- Esto incluye:
  - Términos cuadráticos (e.g., `GrLivArea²`)
  - Interacciones cruzadas (e.g., `GrLivArea * GarageArea`)
- Se evita incluir un término constante (bias)

In [11]:
from sklearn.preprocessing import PolynomialFeatures

# Features seleccionadas
selected_features = strong_features.index.tolist()

# Subset del dataset con esas columnas
X_poly_base = train[selected_features]

# Crear objeto PolynomialFeatures (cuadrático)
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)

# Aplicar transformación
X_poly = poly.fit_transform(X_poly_base)

# Obtener nombres de las nuevas columnas
poly_feature_names = poly.get_feature_names_out(selected_features)

# Convertir a DataFrame para explorarlo
X_poly_df = pd.DataFrame(X_poly, columns=poly_feature_names)

# Mostrar forma
print(f"✅ Forma del dataset original: {X_poly_base.shape}")
print(f"📈 Forma del dataset expandido: {X_poly_df.shape}")


ValueError: Input X contains NaN.
PolynomialFeatures does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values